# Adaptive

[`adaptive`](https://gitlab.kwant-project.org/qt/adaptive-evaluation) is a package for adaptively sampling functions with support for parallel evaluation.

This is an introductory notebook that shows some basic use cases.

`adaptive` needs the following packages:

+ Python 3.6
+ `holoviews`
+ `bokeh`

In [ ]:
import adaptive
adaptive.notebook_extension()

# Import modules that are used in multiple cells
import holoviews as hv
import numpy as np
from functools import partial
import random

# 1D function learner

We start with the most common use-case: sampling a 1D function $\ f: ℝ → ℝ$.

We will use the following function, which is a smooth (linear) background with a sharp peak at a random location:

In [ ]:
offset = random.uniform(-0.5, 0.5)

def f(x, offset=offset, wait=True):
    from time import sleep
    from random import random

    a = 0.01
    if wait:
        sleep(random()*10)
    return x + a**2 / (a**2 + (x - offset)**2)

We start by initializing a 1D "learner", which will suggest points to evaluate, and adapt its suggestions as more and more points are evaluated.

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))

Next we create a "runner" that will request points from the learner and evaluate 'f' on them.

By default the runner will evaluate the points in parallel using local processes ([`concurrent.futures.ProcessPoolExecutor`](https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor)).

In [ ]:
# The end condition is when the "loss" is less than 0.1. In the context of the
# 1D learner this means that we will resolve features in 'func' with width 0.1 or wider.
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)
runner.live_info()

When instantiated in a Jupyter notebook the runner does its job in the background and does not block the IPython kernel.
We can use this to create a plot that updates as new data arrives:

In [ ]:
runner.live_plot()

We can now compare the adaptive sampling to a homogeneous sampling with the same number of points:

In [ ]:
if not runner.task.done():
    raise RuntimeError('Wait for the runner to finish before executing the cells below!')

In [ ]:
learner2 = adaptive.Learner1D(f, bounds=learner.bounds)

xs = np.linspace(*learner.bounds, len(learner.data))
learner2.add_data(xs, map(partial(f, wait=False), xs))

learner.plot() + learner2.plot()

# 2D function learner

Besides 1D functions, we can also learn 2D functions: $\ f: ℝ^2 → ℝ$

In [ ]:
def ring(xy, wait=True):
    import numpy as np
    from time import sleep
    from random import random
    if wait:
        sleep(random())
    x, y = xy
    a = 0.2
    return x + np.exp(-(x**2 + y**2 - 0.75**2)**2/a**4)

learner = adaptive.Learner2D(ring, bounds=[(-1, 1), (-1, 1)])

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)
runner.live_info()

In [ ]:
def plot(learner):
    plot = learner.plot(tri_alpha=0.2)
    title = f'loss={learner._loss:.3f}, n_points={learner.n}'
    opts = dict(plot=dict(title_format=title))
    return plot.Image + plot.EdgePaths.I.clone().opts(**opts) + plot

runner.live_plot(plotter=plot, update_interval=0.2)

In [ ]:
import itertools
learner2 = adaptive.Learner2D(ring, bounds=learner.bounds)
n = int(learner.n**0.5)
xs, ys = [np.linspace(*bounds, n) for bounds in learner.bounds]
xys = list(itertools.product(xs, ys))
learner2.add_data(xys, map(partial(ring, wait=False), xys))
learner2.plot(n).relabel('Homogeneous grid') + learner.plot().relabel('With adaptive')

# Averaging learner

The next type of learner averages a function until the uncertainty in the average meets some condition.

This is useful for sampling a random variable. The function passed to the learner must formally take a single parameter,
which should be used like a "seed" for the (pseudo-) random variable (although in the current implementation the seed parameter can be ignored by the function).

In [ ]:
def g(n):
    import random
    from time import sleep
    sleep(random.random() / 5)
    # Properly save and restore the RNG state
    state = random.getstate()
    random.seed(n)
    val = random.gauss(0.5, 1)
    random.setstate(state)
    return val

In [ ]:
learner = adaptive.AverageLearner(g, atol=None, rtol=0.01)
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 1)
runner.live_info()

In [ ]:
runner.live_plot()

# 1D integration learner with `cquad`

This learner learns a 1D function and calculates the integral and error of the integral with it. It is based on Pedro Gonnet's [implementation](https://www.academia.edu/1976055/Adaptive_quadrature_re-revisited).

Let's try the following function with cusps (that is difficult to integrate):

In [ ]:
def f24(x):
    return np.floor(np.exp(x))

xs = np.linspace(0, 3, 200)
hv.Scatter((xs, [f24(x) for x in xs]))

Just to prove that this really is a difficult to integrate function, let's try a familiar function integrator `scipy.integrate.quad`, which will give us warnings that it encounters difficulties.

In [ ]:
import scipy.integrate
scipy.integrate.quad(f24, 0, 3)

We initialize a learner again and pass the bounds and relative tolerance we want to reach. Then in the `Runner` we pass `goal=lambda l: l.done()` where `learner.done()` is `True` when the relative tolerance has been reached.

In [ ]:
from adaptive.runner import SequentialExecutor
learner = adaptive.IntegratorLearner(f24, bounds=(0, 3), tol=1e-10)
runner = adaptive.Runner(learner, executor=SequentialExecutor(), goal=lambda l: l.done())
runner.live_info()

Now we could do the live plotting again, but lets just wait untill the runner is done.

In [ ]:
if not runner.task.done():
    raise RuntimeError('Wait for the runner to finish before executing the cells below!')

In [ ]:
print('The integral value is {} with the corresponding error of {}'.format(learner.igral, learner.err))
learner.plot()

# 1D learner with vector output: `f:ℝ → ℝ^N`

Some 1D functions return multiple numbers, such as the following function:

In [ ]:
random.seed(0)
offsets = [random.uniform(-0.8, 0.8) for _ in range(3)]
def f_levels(x, offsets=offsets):
    a = 0.01
    return np.array([offset + x + a**2 / (a**2 + (x - offset)**2)
                     for offset in offsets])

This is again a function with sharp peaks at different x-values and with different constant backgrounds. To learn this function we can use a `Learner1D` as well.

In [ ]:
from adaptive.runner import SequentialExecutor

learner = adaptive.Learner1D(f_levels, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=SequentialExecutor(), goal=lambda l: l.loss() < 0.05)

In the plot below we see that the function gets more densely sampled around the peaks, which is the behaviour we want.

In [ ]:
runner.live_plot()

# Balancing learner

The balancing learner is a "meta-learner" that takes a list of multiple leaners. The runner wil find find out which points of which child learner will improve the loss the most and send those to the executor.

The balancing learner can for example be used to implement a poor-man's 2D learner by using the `Learner1D`.

In [ ]:
def f(x, offset):
    a = 0.01
    return x + a**2 / (a**2 + (x - offset)**2)

learners = [adaptive.Learner1D(partial(f, offset=random.uniform(-1, 1)),
            bounds=(-1, 1)) for i in range(10)]

bal_learner = adaptive.BalancingLearner(learners)
runner = adaptive.Runner(bal_learner, goal=lambda l: l.loss() < 0.01)

In [ ]:
plotter = lambda learner: hv.Overlay([L.plot() for L in learner.learners])
runner.live_plot(plotter=plotter)

# DataSaver

Sometimes we want to learn functions that do not only return a single `float`, but instead return multiple values (even though we only want to learn one value.)

We can wrap our learners with a `adaptive.DataSaver` such that the learner will be able to handle functions that return results.

Take for example this function where we want to remember the time that every point took:

In [ ]:
from operator import itemgetter

def f_dict(x):
    """The function evaluation takes roughly the time we `sleep`."""
    import random
    from time import sleep

    waiting_time = random.random()
    sleep(waiting_time)
    a = 0.01
    y = x + a**2 / (a**2 + x**2)
    return {'y': y, 'waiting_time': waiting_time}

# Create the learner with the function that returns a `dict`
# Note that this learner cannot be passed to a runner.
_learner = adaptive.Learner1D(f_dict, bounds=(-1, 1))

# Wrap the learner in the `DataSavingLearner` and tell it which key it needs to learn
learner = adaptive.DataSaver(_learner, arg_picker=itemgetter('y'))

`learner.learner` is the original learner, so `learner.learner.loss()` will call the correct loss method.

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.learner.loss() < 0.01)

Because `learner` doesn't have a `plot` function we need to copy `_learner.plot` (or `learner.learner.plot`)

In [ ]:
learner.plot = _learner.plot
runner.live_plot()

Now the `DataSavingLearner` will have an dictionary attribute `extra_data` that has `x` as key and the data that was returned by `learner.function` as values.

In [ ]:
learner.extra_data

## Alternative executors

Often you will want to evaluate the function on some remote computing resources. `adaptive` works out of the box with any framework that implements a [PEP 3148](https://www.python.org/dev/peps/pep-3148/) compliant executor that returns `concurrent.futures.Future` objects.

### `concurrent.futures`

By default a runner creates a `ProcessPoolExecutor`, but you can also pass one explicitly e.g. to limit the number of workers:

In [ ]:
from concurrent.futures import ProcessPoolExecutor

executor = ProcessPoolExecutor(max_workers=4)

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=executor, goal=lambda l: l.loss() < 0.1)
runner.live_plot()

### IPyparallel

In [ ]:
import ipyparallel

client = ipyparallel.Client()
# f is a closure, so we have to use cloudpickle -- this is independent of 'adaptive'
client[:].use_cloudpickle()

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=client, goal=lambda l: l.loss() < 0.1)
runner.live_plot()

---

# Advanced Topics

## Cancelling a runner

Sometimes you want to interactively explore a parameter space, and want the function to be evaluated at finer and finer resolution and manually control when the calculation stops.

If no `goal` is provided to a runner then the runner will run until cancelled:

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner)
runner.live_plot()

In [ ]:
runner.task.cancel()

## Debugging Problems 

Runners work in the background with respect to the IPython kernel, which makes it convenient, but also means that inspecting errors is more difficult because exceptions will not be raised directly in the notebook. Often the only indication you will have that something has gone wrong is that nothing will be happening.

Let's look at the following example, where the function to be learned will raise an exception 10% of the time.

In [ ]:
def will_raise(x):
    from random import random
    from time import sleep
    
    sleep(random())
    if random() < 0.1:
        raise RuntimeError('something went wrong!')
    return x**2
    
learner = adaptive.Learner1D(will_raise, (-1, 1))
runner = adaptive.Runner(learner)  # without 'goal' the runner will run forever unless cancelled
runner.live_plot()

The above runner should continue forever, but we notice that it stops after a few points are evaluated.

First we should check that the runner has really finished:

In [ ]:
runner.task.done()

If it has indeed finished then we should check the `result` of the runner. This should be `None` if the runner stopped successfully. If the runner stopped due to an exception then asking for the result will raise the exception with the stack trace:

In [ ]:
runner.task.result()

### Logging runners

Runners do their job in the background, which makes introspection quite cumbersome. One way to inspect runners is to instantiate one with `log=True`:

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.1,
                         log=True)
runner.live_plot()

This gives a the runner a `log` attribute, which is a list of the `learner` methods that were called, as well as their arguments. This is useful because executors typically execute their tasks in a non-deterministic order.

This can be used with `adaptive.runner.replay_log` to perfom the same set of operations on another runner:


In [ ]:
reconstructed_learner = adaptive.Learner1D(f, bounds=learner.bounds)
adaptive.runner.replay_log(reconstructed_learner, runner.log)

In [ ]:
learner.plot().opts(style=dict(size=6)) * reconstructed_learner.plot()

### Timing functions

To time the runner you **cannot** simply use 
```python
now = datetime.now()
runner = adaptive.Runner(...)
print(datetime.now() - now)
```
because this will be done immediately. Also blocking the kernel with `while not runner.task.done()` will not work because the runner will not do anything when the kernel is blocked.

Therefore you need to create an `async` function and hook it into the `ioloop` like so:

In [ ]:
import asyncio

async def time(runner):
    from datetime import datetime
    now = datetime.now()
    await runner.task
    return datetime.now() - now

ioloop = asyncio.get_event_loop()

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)

timer = ioloop.create_task(time(runner))

In [ ]:
# The result will only be set when the runner is done.
timer.result()

## Using Runners from a script 

Runners can also be used from a Python script independently of the notebook:

```python
import adaptive

def f(x):
    return x

learner = adaptive.Learner1D(f, (-1, 1))

runner = adaptive.Runner(learner, goal=lambda: l: l.loss() < 0.1)
runner.run_sync()  # Block until completion.
```

Under the hood the runner uses [`asyncio`](https://docs.python.org/3/library/asyncio.html). You don't need to worry about this most of the time, unless your script uses asyncio itself. If this is the case you should be aware that instantiating a `Runner` schedules a new task on the current event loop, and you can simply

```python
    await runner.task
```

inside a coroutine to await completion of the runner.